In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [6]:
def left_right_lines(hough_lines):
    #bottom left x,y
    #left min x, max y
    left_x1=10000
    left_y1=0
    #top left x,y
    left_x2=0
    left_y2=10000
    # top right min x, min y
    right_x1=10000
    right_y1=10000
    #bottom left
    right_x2=0
    right_y2=0
    for i in hough_lines:
        if (i[0,0] < 480):
            if i[0,0] < left_x1:
                left_x1=i[0,0]       
            
            if i[0,1] > left_y1:
                left_y1=i[0,1]
            
            if i[0,2] > left_x2:
                left_x2=i[0,2]
            
            if i[0,3] < left_y2:
                left_y2=i[0,3]
        else:
            if i[0,0] < right_x1:
                right_x1=i[0,0]
            if i[0,1] < right_y1:
                right_y1=i[0,1]
            if i[0,2] > right_x2:
                right_x2=i[0,2]
            if i[0,3] > right_y2:
                right_y2=i[0,3]
    
    final_lines=np.ndarray(shape=(2,4), dtype=int) #, order='F')
    final_lines = [[left_x1,left_y1,left_x2, left_y2],
                          [right_x1, right_y1, right_x2,right_y2]]
    return final_lines

In [7]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    l_r_lines=left_right_lines(lines)
    left_line=l_r_lines[0]
    right_line=l_r_lines[1]
    cv2.line(img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), [255, 0,0], 3)
    cv2.line(img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), [255, 0,0], 3)
    

In [8]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

In [9]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [10]:
def process_image(image):
    ## system parameters for number of functions
    gaus_kernel_size=5
    #Canny edge detection parameters
    canny_low_th= 50
    canny_high_th = 150
    #Define masking region
    #image dimentions y=540, x=960
    imshape=[540, 960]
    left_xbottom=0
    left_ybottom=imshape[0]
    
    left_xtop=460
    left_ytop=322
    
    right_xtop=490
    right_ytop=left_ytop
    
    right_xbottom=imshape[1]
    right_ybottom=left_ybottom
    im_vertices = np.array([[(left_xbottom,left_ybottom),(left_xtop, left_ytop), (right_xtop, right_ytop), (right_xbottom,right_ybottom)]], dtype=np.int32)
    # Make a blank the same size as our image to draw on
    hgh_rho = 2 # 1 distance resolution in pixels of the Hough grid
    hgh_theta = np.pi/180 # angular resolution in radians of the Hough grid
    hgh_th = 10     # minimum number of votes (intersections in Hough grid cell)
    hgh_min_line_l = 15 # 5 minimum number of pixels making up a line
    hgh_max_line_gap = 10    # maximum gap in pixels between connectable line segments
    #line_image = np.copy(image)*0 # creating a blank to draw lines on
#    im_cp= np.copy(image)
    #printing out some stats and plotting
    #print('This image is:', type(image), 'with dimesions:', image.shape)
    #use gray image
    gray_im=grayscale(image)
    #gaussian blur
    gray_im=gaussian_blur(gray_im, gaus_kernel_size)
    #find edges
    gray_im=canny(gray_im, canny_low_th, canny_high_th)
    #mask the region of interest                   
    gray_im=region_of_interest(gray_im, im_vertices)
    #find the hough line and apply to the image
    gray_im=hough_lines(gray_im, hgh_rho, hgh_theta, hgh_th, hgh_min_line_l, hgh_max_line_gap)
    # compbine everything
    gray_im=weighted_img(image, gray_im, α=0.7, β=1., λ=0.)
    
    #make single lines left and right
    

    return gray_im

In [11]:
white_output='white.mp4'
clip1=VideoFileClip("solidWhiteRight.mp4")
white_clip=clip1.fl_image(process_image)
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:07<00:00, 28.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 



In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [13]:
yellow_output='yellow.mp4'
clip1=VideoFileClip("solidYellowLeft.mp4")
white_clip=clip1.fl_image(process_image)
white_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:29<00:00, 20.12it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 



In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [15]:
challenge_output='challenge_out.mp4'
clip1=VideoFileClip("challenge.mp4")
white_clip=clip1.fl_image(process_image)
white_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challenge_out.mp4
[MoviePy] Writing video challenge_out.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:25<00:00, 10.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_out.mp4 



In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))